In [1]:
###### check two data set
import os, sys
sys.path.append("/home/n/nguyenpk/CS6220/otdd")
import otdd
from otdd.pytorch.distance import DatasetDistance

/home/n/nguyenpk/CS6220/otdd/otdd/pytorch/distance.py:23: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-04-16 01:02:52.318209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
ot.gpu not found - coupling computation will be in cpu


In [6]:
!ls data

Daily_Dailog_raw.pkl		       IEMOCAP_embedd.pkl
Daily_embedd.pkl		       IEMOCAP_raw.pkl
Daily_token_fts.pkl		       IEMOCAP_token_fts.pkl
Diaglog_raw.pkl			       MELD_embedd.pkl
Glove_embedding_Daily_token_fts.pkl    MELD_raw.pkl
Glove_embedding_IEMOCAP_token_fts.pkl  MELD_token_fts.pkl
Glove_embedding_MELD_token_fts.pkl     preprocessing


In [7]:
Dataset = {}
for i in ["IEMOCAP", "Daily", "MELD"]:
    Dataset[i] = pd.read_pickle("data/Glove_embedding_{}_token_fts.pkl".format(i))
#####


In [8]:
i = "Daily"
idata = Dataset[i]

In [16]:
idata["train"]["topics"]

[]

In [2]:
import math
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torch.nn.utils.rnn import pad_sequence
import pickle, pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import torch.nn as nn

import torch
import numpy as np


seed = 100
if torch.cuda.is_available():
    generator = torch.Generator('cuda').manual_seed(seed)
else:
    generator = torch.Generator().manual_seed(seed)


class my_DataSet(Dataset):
    """
    """
    def __init__(self, path , split, pretrain_emb=None):
        data = pd.read_pickle(path)[split]
        Xdata = torch.tensor(np.concatenate(data["data_token"]), dtype=torch.int)
        self.data = pretrain_emb(Xdata).sum(axis=1)
        # self.clf = preprocessing.LabelEncoder()
        y = np.concatenate(data["emotions"])
        # self.targets = torch.tensor(self.clf.fit_transform(y), dtype=torch.int)
        # self.data = Xdata
        self.targets= torch.tensor(y, dtype=torch.int)
        self.len = len(self.targets)
    def __getitem__(self, index):
        return self.data[index], self.targets[index]
    def __len__(self):
        return self.len
    def __delitem__(self, key):
        self.data = np.delete(self.data, key, axis=0)
        self.targets = np.delete(self.targets, key, axis=0)
        

In [26]:
import os
os.environ['CUDA_PATH'] = '/usr/local/cuda-11'

In [29]:
dist1 = DatasetDistance(train, test,
                           inner_ot_method = 'gaussian_approx',
                           debiased_loss = True,
                           p = 2, entreg = 1e-1,
                           min_labelcount=1,
                           device='cuda:0')
d = dist1.distance()
d = dist1.distance(maxsamples = 100)

0it [00:00, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]